a, b と改行で並べるだけで出力できるように設定します

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

`data/input/train.csv` のデータを元に主要な統計量を計算します。

In [ ]:
from numpy import test
import pandas as pd
import sys
from pathlib import Path
# codeフォルダへのパスを追加
sys.path.append(str(Path("..").resolve()))
from python.path_utils import input_path

# パス設定
INPUT_PATH = input_path("train.csv")
TEST_PATH = input_path("test.csv")

# データ読み込み
df = pd.read_csv(INPUT_PATH)
test_df = pd.read_csv(TEST_PATH)

dfの内容を詳しく見ます。

In [ ]:
# 構造の把握
df
df.shape          # (行数, 列数)
df.dtypes         # 各列の型
df.info()         # 型 + 非null数 + メモリ使用量を一覧表示
df.describe()     # 数値列の統計量（count, mean, std, min, max, 四分位）

# データの中身を見る
df.head()         # 先頭5行（引数で行数指定可）
df.tail(10)       # 末尾10行
df.sample(5)      # ランダムに5行（偏りなく雰囲気を掴める）

# 欠損・ユニーク値
df.isnull().sum()          # 列ごとの欠損数
df["Age"].value_counts()   # カテゴリ列の分布
df.nunique()               # 列ごとのユニーク値数

Age・cabin・embarked列には欠損値があることがわかります。
それらのカラムのデータ型を確認しましょう。

In [ ]:
df.dtypes[df.isnull().sum() > 0]

Ageは数値列、cabinとembarkedは文字列列であることがわかります。
Age列は数値が入っているため、平均や中央値などの統計量を計算できます。

In [ ]:
df.nunique()[df.isnull().sum() > 0]

文字列型のCabinとEmbarkedですが、Embarkedはユニークな値が3つしかないため、カテゴリ列として扱うことができます。
ひとまず、最も多い値で欠損値を埋めることにしましょう。

In [ ]:
df["Embarked"].unique()

In [ ]:
# Embarkedの最頻値で欠損を補完しカテゴリ型に変換
embarked_mode = df["Embarked"].mode(dropna=True)[0]
# 欠損値を最頻値で埋めてからカテゴリ型に変換
# カテゴリ型に変換することで、モデルが文字列を数値として扱えるようになります。（ENUM型のようなもの）
df["Embarked"] = df["Embarked"].fillna(embarked_mode).astype("category")

# 確認
embarked_mode
df["Embarked"].isnull().sum()
df["Embarked"].value_counts()

In [ ]:
# Ageの平均値で欠損を補完
age_mean = df["Age"].mean()
df["Age"] = df["Age"].fillna(age_mean)

# 確認
age_mean
df["Age"].isnull().sum()
df["Age"].describe()

Cabin列はユニークな値も多く、欠損値も多いため、今回は削除してしまいましょう。

In [ ]:
# Cabin列を削除
df = df.drop(columns=["Cabin"] )

# 確認
"Cabin" in df.columns, df.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# 元データからモデルをトレーニング
y = df["Survived"]
features = ["Age", "Sex"]
df["Sex"] = df["Sex"].astype("category")  # 文字列をカテゴリ型に変換

# EmbarkedはすでにCategory型なので、cat.codesで整数に変換する
# get_dummies（ワンホットエンコーディング）だと Embarked_C <= 0.5 のような
# 「実質 == 0 か？」という読みにくい条件が生まれてしまうため
X = df[features].copy()
X["Sex"] = X["Sex"].cat.codes  # male=0, female=1 のような整数に変換

# criterionにはgini（ジニ不純度）とentropy（エントロピー）があります。どちらも大きな差はないことが多いため、今回はginiを使用します。
model = DecisionTreeClassifier(max_depth=4, criterion='gini', min_samples_leaf=5, max_leaf_nodes=8, random_state=42)
model.fit(X, y)

print(f"Train: {model.score(X, y):.3f}")
print(f"\nSexのコード対応: {dict(enumerate(df['Sex'].cat.categories))}")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

plt.figure(figsize=(16, 8))
plot_tree(model, feature_names=X.columns,
          class_names=["Not Survived", "Survived"], filled=True, rounded=True)
plt.tight_layout()
# plt.savefig("tree.png", dpi=150)
plt.show()

In [ ]:
path = model.cost_complexity_pruning_path(X, y)
ccp_alphas = path.ccp_alphas
impurities = path.impurities
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 4))
plt.plot(ccp_alphas, impurities, marker="o", drawstyle="steps-post")
plt.xlabel("ccp_alpha")
plt.ylabel("impurity")
plt.title("Cost-Complexity Pruning Path")
plt.show()

In [ ]:
# Step 2: 各alphaで木を構築してスコアを比較
train_scores, test_scores = [], []
trees = []
for alpha in ccp_alphas:
    clf = DecisionTreeClassifier(ccp_alpha=alpha, random_state=42)
    clf.fit(X, y)
    train_scores.append(clf.score(X, y))
    test_scores.append(clf.score(X, y))
    trees.append(clf)

# Step 3: 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# alpha vs 精度
axes[0].plot(ccp_alphas, train_scores, marker='o', label='train', markersize=3)
axes[0].plot(ccp_alphas, test_scores, marker='o', label='test', markersize=3)
axes[0].set_xlabel('ccp_alpha')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy vs alpha')
axes[0].legend()

# alpha vs ノード数
node_counts = [t.tree_.node_count for t in trees]
axes[1].plot(ccp_alphas, node_counts, marker='o', markersize=3)
axes[1].set_xlabel('ccp_alpha')
axes[1].set_ylabel('Number of nodes')
axes[1].set_title('Tree size vs alpha')

plt.tight_layout()
plt.savefig('ccp_analysis.png', dpi=150)
plt.show()

In [ ]:
for alpha in ccp_alphas:
    print(f"alpha: {alpha:.5f}")

In [ ]:
df["Sex"][0]

In [ ]:
print(df["Sex"].cat.codes)